[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/LDA_content.ipynb)

# Latent Dirichlet Allocation (LDA)

In [3]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

# データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

In [1]:
# Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [2]:
# 因子数
factors = 50
# エポック数
n_epochs = 30

In [3]:
movie_content = movies.copy()
# tagが付与されていない映画もあるが、genreはすべての映画に付与されている
# tagとgenreを結合したものを映画のコンテンツ情報として似ている映画を探して推薦していく
# tagがない映画に関しては、NaNになっているので、空のリストに変換してから処理をする
movie_content["tag_genre"] = movie_content["tag"].fillna("").apply(list) + movie_content["genre"].apply(list)
movie_content["tag_genre"] = movie_content["tag_genre"].apply(lambda x: list(map(str, x)))

# タグとジャンルデータを使って、ldaを学習する
tag_genre_data = movie_content.tag_genre.tolist()

In [ ]:
!pip install gensim==4.0.1

In [4]:
from gensim.corpora.dictionary import Dictionary

# LDAのインプットに使うデータの作成
common_dictionary = Dictionary(tag_genre_data)
common_corpus = [common_dictionary.doc2bow(text) for text in tag_genre_data]


/Users/mk/.pyenv/versions/3.7.8/envs/recsys-test3.7.8/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [6]:
import gensim

# LDAの学習
lda_model = gensim.models.LdaModel(
    common_corpus, id2word=common_dictionary, num_topics=factors, passes=n_epochs
)

In [17]:
# topic0の単語一覧
for token_id, score in lda_model.get_topic_terms(0, topn=10):
    word = common_dictionary.id2token[token_id]
    print(f'word={word}, score={score}')

word=drugs, score=0.12014076858758926
word=japan, score=0.0761302188038826
word=quentin tarantino, score=0.05024522915482521
word=will smith, score=0.03854871168732643
word=tarantino, score=0.029550844803452492
word=tommy lee jones, score=0.02945614792406559
word=akira kurosawa, score=0.028017211705446243
word=samurai, score=0.025930559262633324
word=ummarti2007, score=0.02379794418811798
word=addiction, score=0.018428761512041092


In [21]:
# サムライという単語のトピック(各トピックの所属確率)
lda_model[common_dictionary.doc2bow(['samurai'])]

[(0, 0.509997),
 (1, 0.010000061),
 (2, 0.010000061),
 (3, 0.010000061),
 (4, 0.010000061),
 (5, 0.010000061),
 (6, 0.010000061),
 (7, 0.010000061),
 (8, 0.010000061),
 (9, 0.010000061),
 (10, 0.010000061),
 (11, 0.010000061),
 (12, 0.010000061),
 (13, 0.010000061),
 (14, 0.010000061),
 (15, 0.010000061),
 (16, 0.010000061),
 (17, 0.010000061),
 (18, 0.010000061),
 (19, 0.010000061),
 (20, 0.010000061),
 (21, 0.010000061),
 (22, 0.010000061),
 (23, 0.010000061),
 (24, 0.010000061),
 (25, 0.010000061),
 (26, 0.010000061),
 (27, 0.010000061),
 (28, 0.010000061),
 (29, 0.010000061),
 (30, 0.010000061),
 (31, 0.010000061),
 (32, 0.010000061),
 (33, 0.010000061),
 (34, 0.010000061),
 (35, 0.010000061),
 (36, 0.010000061),
 (37, 0.010000061),
 (38, 0.010000061),
 (39, 0.010000061),
 (40, 0.010000061),
 (41, 0.010000061),
 (42, 0.010000061),
 (43, 0.010000061),
 (44, 0.010000061),
 (45, 0.010000061),
 (46, 0.010000061),
 (47, 0.010000061),
 (48, 0.010000061),
 (49, 0.010000061)]

In [22]:
# 各映画のトピックを格納
lda_topics = lda_model[common_corpus]

# 各映画に最も確率の高いトピックを１つ取り出し格納していく
movie_topics = []
movie_topic_scores = []
for movie_index, lda_topic in enumerate(lda_topics):
    sorted_topic = sorted(lda_topics[movie_index], key=lambda x: -x[1])
    # 最も確率の高いトピック
    movie_topic, topic_score = sorted_topic[0]
    movie_topics.append(movie_topic)
    movie_topic_scores.append(topic_score)
movie_content["topic"] = movie_topics
movie_content["topic_score"] = movie_topic_scores
movie_content

,movie_id,title,genre,tag,tag_genre,topic,topic_score
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[pixar, pixar, pixar, animation, pixar, animat...","[pixar, pixar, pixar, animation, pixar, animat...",20,0.642943
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[for children, game, animals, joe johnston, ro...","[for children, game, animals, joe johnston, ro...",1,0.473445
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[funniest movies, comedinha de velhinhos engra...","[funniest movies, comedinha de velhinhos engra...",38,0.601124
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",[girl movie],"[girl movie, Comedy, Drama, Romance]",33,0.755000
4,5,Father of the Bride Part II (1995),[Comedy],"[steve martin, pregnancy, remake, steve martin...","[steve martin, pregnancy, remake, steve martin...",21,0.600993
...,...,...,...,...,...,...,...
10676,65088,Bedtime Stories (2008),"[Adventure, Children, Comedy]",NaN,"[Adventure, Children, Comedy]",1,0.755000
10677,65091,Manhattan Melodrama (1934),"[Crime, Drama, Romance]",NaN,"[Crime, Drama, Romance]",33,0.388229
10678,65126,Choke (2008),"[Comedy, Drama]","[chuck palahniuk, based on book]","[chuck palahniuk, based on book, Comedy, Drama]",33,0.505002
10679,65130,Revolutionary Road (2008),"[Drama, Romance]",[toplist08],"[toplist08, Drama, Romance]",33,0.673333


In [30]:
from collections import defaultdict
from collections import Counter

# 各ユーザーのレコメンドリストを作成していく
# ユーザーが高く評価した映画が、どのトピックに所属していることが多いかをカウントする
# 一番多いトピックをユーザーの好きなトピックとみなして、そのトピックの映画をおすすめする

movielens_train_high_rating = movielens_train[movielens_train.rating >= 4]
user_evaluated_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()

movie_id2index = dict(zip(movie_content.movie_id.tolist(), range(len(movie_content))))
pred_user2items = defaultdict(list)
for user_id, data in movielens_train_high_rating.groupby("user_id"):
    # ユーザーが高く評価した映画
    evaluated_movie_ids = user_evaluated_movies[user_id]
    # 直近閲覧した映画を取得
    movie_ids = data.sort_values("timestamp")["movie_id"].tolist()[-10:]

    movie_indexes = [movie_id2index[id] for id in movie_ids]
    
    # 直近閲覧した映画のトピックを取得して、出現回数をカウントする
    topic_counter = Counter([movie_topics[i] for i in movie_indexes])
    # 一番出現回数が多かったトピックを取得
    frequent_topic = topic_counter.most_common(1)[0][0]
    # そのトピックの映画の中でもスコアが高いものをおすすめする
    topic_movies = (
        movie_content[movie_content.topic == frequent_topic]
        .sort_values("topic_score", ascending=False)
        .movie_id.tolist()
    )

    for movie_id in topic_movies:
        if movie_id not in evaluated_movie_ids:
            pred_user2items[user_id].append(movie_id)
        if len(pred_user2items[user_id]) == 10:
            break
pred_user2items

defaultdict(list,
            {1: [3759, 3964, 3775, 1022, 1030, 2096, 48, 2687, 6414, 2566],
             2: [2115, 76, 2006, 1587, 1779, 8644, 4643, 377, 6934, 2848],
             3: [150, 2375, 4022, 993, 5341, 4468, 7456, 4192, 2501, 2822],
             4: [2115, 76, 2006, 1587, 1779, 8644, 4643, 6934, 2848, 1198],
             5: [1256, 3742, 7091, 913, 25777, 1358, 1186, 34002, 8734, 25766],
             6: [6876, 4273, 2432, 8451, 7372, 6641, 5408, 7761, 5533, 1624],
             7: [3742,
              7091,
              25777,
              1358,
              1186,
              34002,
              8734,
              25766,
              34018,
              7132],
             8: [5780, 4687, 3693, 4928, 193, 4532, 2897, 7577, 5604, 26915],
             9: [1223, 1148, 1020, 6104, 745, 788, 6807, 2788, 38038, 1136],
             10: [3100, 1933, 1929, 1935, 4292, 1946, 1943, 8153, 7771, 1937],
             11: [2204, 2178, 2185, 2179, 1269, 930, 949, 903, 933, 904],
     

In [31]:
# user_id=2のユーザーが学習データで、4以上の評価を付けた映画一覧
movielens_train_high_rating[movielens_train_high_rating.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
8381,2,1210,4.0,868245644,Star Wars: Episode VI - Return of the Jedi (1983),"[Action, Adventure, Sci-Fi]","[desert, fantasy, sci-fi, space, lucas, gfei o...",10.0


In [32]:
# user_id=2に対するおすすめ(2115, 76, 2006)
movies[movies.movie_id.isin([2115, 76, 2006])]

,movie_id,title,genre,tag
75,76,Screamers (1995),"[Action, Sci-Fi, Thriller]","[philip k. dick, artificial intelligence, post..."
1922,2006,"Mask of Zorro, The (1998)","[Action, Adventure, Romance]","[california, mexico, funny, banderas, anthony ..."
2031,2115,Indiana Jones and the Temple of Doom (1984),"[Action, Adventure]","[lucas, want it, dvd collection, harrison ford..."


In [ ]:
# 今回は、各ユーザーの評価履歴から１つのトピックをピックアップしましたが、トピックの確率値を利用して、重み付けしながら、アイテムを抽出することも可能です